In [0]:
%run ./config

## 1. 노트북에서 데이터를 쿼리하고 시각화하기

In [0]:
%sql
SELECT * FROM samples.nyctaxi.trips

In [0]:
%python
display(spark.read.table("samples.nyctaxi.trips"))

Databricks visualization. Run in Databricks to view.

Databricks data profile. Run in Databricks to view.

## 데이터 시각화

* Next to the **Table** tab, click + and then click Visualization.

* In the **Visualization Type** drop-down, verify that **Bar** is selected.

* Select fare_amount for the **X column**.

* Select trip_distance for the **Y column**.

* Select Average as the aggregation type.

* Select pickup_zip as the **Group by** column.

![시각화 구성](https://docs.databricks.com/aws/en/assets/images/trip_distance-79aa375bd64d7573daf37b63f59d173a.png)

### AI Assistant 활용하기

In [0]:
%sql
-- 날짜별 요금 집계 (SQL)

In [0]:
%python
# 요일별 운행 거리 총합

## 2. CSV 데이터 임포트하고 시각화하기

In [0]:
display(spark.sql("SHOW CURRENT SCHEMA"))

In [0]:
# sample data download 
download_url = "https://health.data.ny.gov/api/views/jxy9-yhdk/rows.csv"
file_name = "baby_names.csv"
table_name = "baby_names"
path_volume = "/Volumes/" + catalog + "/" + schema + "/" + volume
path_table = catalog + "." + schema
print(path_table) # Show the complete path
print(path_volume) # Show the complete path

In [0]:
dbutils.fs.cp(f"{download_url}", f"{path_volume}" + "/" + f"{file_name}")

In [0]:
dbutils.fs.ls(f"{path_volume}" + "/" + f"{file_name}") 

In [0]:
%fs head /Volumes/demo_ykko/product_tour/vol_product_tour/baby_names.csv

In [0]:
df = spark.read.csv(f"{path_volume}/{file_name}",
  header=True,
  inferSchema=True,
  sep=",")

In [0]:
display(df)

Databricks visualization. Run in Databricks to view.

In [0]:
# 컬럼명 규칙에 맞게 수정
df = df.withColumnRenamed("First Name", "First_Name")
df.printSchema

In [0]:
df.write.mode("overwrite").saveAsTable(f"{path_table}" + "." + f"{table_name}")

In [0]:
%sql
SELECT * FROM baby_names;

In [0]:
%sql
CREATE OR REPLACE TABLE aggr_by_year_county AS 
SELECT `Year`, County, sum(Count) as total 
FROM baby_names
GROUP BY `Year`, County
ORDER BY total DESC;

In [0]:
%sql
CREATE OR REPLACE TABLE aggr_by_year_fname AS 
SELECT `Year`, First_Name, sum(Count) as total 
FROM baby_names
GROUP BY `Year`, First_Name
ORDER BY total DESC;

In [0]:
%sql
CREATE OR REPLACE TABLE aggr_by_year AS 
SELECT a.`Year`, count(distinct a.County) as uniq_county, 
  count(distinct b.First_Name) as uniq_fname
FROM aggr_by_year_county a
LEFT OUTER JOIN aggr_by_year_fname b
ON a.`Year` = b.`Year`
GROUP BY a.`Year`

In [0]:
%sql SELECT * FROM aggr_by_year

## 3. 데이터 관리하기 
* Table 권한 설정
* AI Generated comment 로 테이블 메타데이터 추가
* 리니지 

## 4. Lakehouse Federation 으로 원격 데이터 조회

In [0]:
%sql
-- BigQuery 에서 실시간 데이터 조회 
-- SELECT * FROM `tania-bq-federation-test`.abhijeet_rao.20k_urls
-- LIMIT 100

-- SQL Server 에서 실시간 데이터 조회
SELECT * FROM mssql_foreigncatalog.ak.orders LIMIT 100;


## 5. 대시보드 (with Genie)
* Dashboard > Gallery > New York Taxi 

## 6. SQL 에서 AI 기능 사용 (AI functions)

In [0]:
%sql
SELECT * FROM samples.bakehouse.media_customer_reviews LIMIT 5;

In [0]:
%sql
SELECT review,
  ai_analyze_sentiment(review) AS sentiment
FROM samples.bakehouse.media_customer_reviews
LIMIT 10;

In [0]:
%sql
SELECT
  review,
  ai_gen(
    "한글 60자 이내로 고객의 리뷰에 대한 정중한 응답 메시지를 생성하시오. 리뷰의 불만사항을 언급하고 해결을 위해 노력하겠다는 내용을 포함하세요. 고객 리뷰 : " || review
  ) AS reply
FROM samples.bakehouse.media_customer_reviews
WHERE ai_analyze_sentiment(review) = "negative"
LIMIT 5;

## Genie space